In [14]:
from keras.layers import *
from keras.models import Model
from keras import optimizers
import numpy as np
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import os
from keras import backend as K


shape_range =range(96,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    
    inputs = tf.keras.Input(shape=(shape_x,shape_y,1))
    x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_classifier')
    model.summary()
    print(len(model.layers))
    #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #history = model.fit(X_train, Y_train, epochs=1)
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
    --model ./quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_simple \
    --net_name deploy
    os.rename("./xmodel_simple/deploy.xmodel","./xmodel_simple/simple_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))



Model: "mnist_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 45, 45, 32)        9248      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 15488)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 1024)         

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_simple/deploy.xmodel' -> './xmodel_simple/simple_96_96.xmodel'